### **State University of Campinas - UNICAMP** </br>
**Course**: MC886A </br>
**Professor**: Marcelo da Silva Reis </br>
**TA (PED)**: Marcos Vinicius Souza Freire

---

### **Hands-On: Logistic Regression, Classification Methods, and Resampling Methods**
##### Notebook: 00 Logistic Regression and Classification and Resampling methods

> Dataset from Scikit Learn - [load_breast_cancer](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_breast_cancer.html), based on [Breast Cancer Wisconsin (Diagnostic)](https://archive.ics.uci.edu/dataset/17/breast+cancer+wisconsin+diagnostic)(1993)[1]
---

**This notebook covers the following topics:**

- **Logistic Regression:** binary, multiple, and multinomial extensions.
- **Classification Methods:** Linear Discriminant Analysis (LDA), Quadratic Discriminant Analysis (QDA) and Naive Bayes.
- **Resampling Methods:** Leave-One-Out (LOOCV), k-Fold Cross-Validation and Bootstrap.

Throughout the notebook we illustrate the methods using formulas, interactive Plotly graphs for the decision boundaries, and well-structured code cells.

---

### **Notation and Formulas**


### **1. Binary Logistic Regression**  
Used for **two-class classification** (e.g., yes/no, 0/1).

#### **Formula**:  
The probability $ p $ that an instance belongs to class $ y=1 $ is modeled using the **sigmoid function**:  
$
p(y=1 \mid \mathbf{x}) = \frac{1}{1 + e^{-z}}, \quad \text{where } z = \beta_0 + \beta_1 x_1 + \beta_2 x_2 + \dots + \beta_n x_n
$  
- $ \mathbf{x} = [x_1, x_2, \dots, x_n] $: Input features.  
- $ \beta_0, \beta_1, \dots, \beta_n $: Model coefficients.  
- **Decision**: Classify as $ y=1 $ if $ p \geq 0.5 $, else $ y=0 $.  

#### **Logit (Log-Odds)**:  
$
\ln\left(\frac{p}{1-p}\right) = \beta_0 + \beta_1 x_1 + \dots + \beta_n x_n
$  


This linear equation represents the relationship between features and the log-odds of the positive class.

</br>

#### **Loss Function (Cross-Entropy Loss)**:  
$
\mathcal{L} = -\frac{1}{N} \sum_{i=1}^N \left[ y_i \ln(p_i) + (1-y_i) \ln(1-p_i) \right]
$  

</br>

Minimizing this loss adjusts the coefficients $ \beta $ via gradient descent or MLE.

---
</br>

### **2. Multinomial Logistic Regression**  
Used for **multi-class classification** (e.g., classes A/B/C/D).

#### **Main Formula**:  
The probability $ p(y=k \mid \mathbf{x}) $ for class $ k $ is modeled using the **softmax function**:  
$
p(y=k \mid \mathbf{x}) = \frac{e^{z_k}}{\sum_{j=1}^K e^{z_j}}, \quad \text{where } z_k = \beta_{k0} + \beta_{k1} x_1 + \dots + \beta_{kn} x_n
$  
- $ K $: Total number of classes.  
- $ z_k $: Linear combination for class $ k $.  
- **Decision**: Classify as the class with the highest probability $ p(y=k \mid \mathbf{x}) $.  

#### **Key Notes**:  
- One class (e.g., $ K $) is typically treated as the **reference category**, and its coefficients are set to zero (e.g., $ z_K = 0 $).  
- The model estimates $ K-1 $ sets of coefficients.  

#### **Loss Function (Generalized Cross-Entropy)**:  
$
\mathcal{L} = -\frac{1}{N} \sum_{i=1}^N \sum_{k=1}^K y_{ik} \ln(p_{ik})
$  
- $ y_{ik} = 1 $ if observation $ i $ is in class $ k $, else 0.  
- $ p_{ik} $: Predicted probability that observation $ i $ belongs to class $ k $.  

---

#### **Differences**  
| **Aspect**              | **Binary**                          | **Multinomial**                     |  
|--------------------------|-------------------------------------|-------------------------------------|  
| **Classes**              | 2 classes (0/1)                    | $ K \geq 2 $ classes              |  
| **Function**             | Sigmoid                            | Softmax                             |  
| **Coefficients**         | One set ($ \beta_0, \beta_1, \dots $) | $ K-1 $ sets (one per class) |  

---

### **Example Applications**  
1. **Binary**:  
   - Predict if an email is spam ($ p \geq 0.5 $) or not.  
   - Compute $ z = 2.5 + 0.8x_1 - 1.2x_2 $, then $ p = \frac{1}{1 + e^{-z}} $.  

2. **Multinomial**:  
   - Classify an image into "cat," "dog," or "bird."  
   - For features $ \mathbf{x} $, compute probabilities:  
     $
     p(\text{cat}) = \frac{e^{z_{\text{cat}}}}{e^{z_{\text{cat}}} + e^{z_{\text{dog}}} + e^{z_{\text{bird}}}}
     $  
     (Similarly for other classes.)


Based on the Jurafsky & Martin (2025) lectures [2]

---


In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd

# Replace Matplotlib with Plotly for interactive plotting
import plotly.graph_objects as go
import plotly.express as px

from sklearn.datasets import make_classification, load_breast_cancer
from sklearn.model_selection import train_test_split, KFold, LeaveOneOut, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset

import warnings
warnings.filterwarnings('ignore')

# Set seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)


#### **Basic exploration of the dataset**

In [ ]:
# Let's load the Breast Cancer Dataset from Scikit-Learn
cancer_dataset = load_breast_cancer()

In [ ]:
# Keys in dataset
cancer_dataset.keys()

In [ ]:
# Malignant or benign value
cancer_dataset['target']

In [ ]:
# Target value name malignant or benign tumor
cancer_dataset['target_names']

In [ ]:
# Description of data
print(cancer_dataset['DESCR'])

In [ ]:
# Name of features
print(cancer_dataset['feature_names'])

In [ ]:
# Create datafrmae
cancer_df = pd.DataFrame(np.c_[cancer_dataset['data'],cancer_dataset['target']],
             columns = np.append(cancer_dataset['feature_names'], ['target']))

In [ ]:
# Head of cancer DataFrame
cancer_df.head(6)

In [ ]:
# Tail of cancer DataFrame
cancer_df.tail(6)

In [ ]:
# Information of cancer Dataframe
cancer_df.info()

In [ ]:
# Numerical distribution of data
cancer_df.describe()

---

### **Helper Functions**

In this section, we define helper functions to evaluate classifiers and to plot decision boundaries.

**Decision Boundary Plotting with Plotly:**

We create a function (`plot_decision_boundary_plotly`) that plots the decision boundary using Plotly. For a given model, we generate a grid over the feature space, predict the classes for each grid point, and then plot a contour along with the data points.

Also, we implement the `evaluate_classifier`, which prints:

1. **Accuracy**
   - **What It Measures**: Accuracy is the proportion of predictions the classifier got right out of all predictions made, i.e., the proportion of correctly classified instances.
   - **Formula**:
   $
   \text{Accuracy} = \frac{\text{Number of Correct Predictions}}{\text{Total Number of Predictions}}
   $
   - **Output**: A single number between 0 and 1 (e.g., `0.85` means 85% of predictions were correct).
   - **Why It Matters**: It gives a quick representation of overall performance but can be misleading if your dataset has imbalanced classes (e.g., 90% of one class and 10% of another).

2. **Confusion Matrix**
   - **What It Shows**: This is a table that counts how many times the classifier predicted each class correctly or incorrectly compared to the true labels.

   - **Structure** (for binary classification):

|                | Predicted Positive | Predicted Negative |
|----------------|--------------------|--------------------|
| **Actual Positive** | True Positives (TP) | False Negatives (FN) |
| **Actual Negative** | False Positives (FP) | True Negatives (TN)  |

   - **Example Output**:
     ```
     [[50  5]
      [10 35]]
     ```
     Here, 50 true negatives, 5 false positives, 10 false negatives, and 35 true positives.
   - **Why It Matters**: It reveals the specific types of errors (e.g., mistaking positives for negatives), which is crucial for understanding model behavior beyond just accuracy.

3. **Classification Report**
   - **What It Provides**: A detailed summary of performance for each class, including:
     - **Precision**: How many of the predicted positives are actually positive.
       $
       \text{Precision} = \frac{\text{TP}}{\text{TP} + \text{FP}}
       $
     - **Recall**: How many of the actual positives were correctly predicted.
       $
       \text{Recall} = \frac{\text{TP}}{\text{TP} + \text{FN}}
       $
     - **F1-Score**: A balanced measure combining precision and recall.
       $
       \text{F1-Score} = 2 \times \frac{\text{Precision} \times \text{Recall}}{\text{Precision} + \text{Recall}}
       $
     - **Support**: The number of true instances (number of occurrences) of each class in the dataset.
   - **Example Output**:
     ```
                  precision    recall  f1-score   support
            0       0.83      0.91      0.87        55
            1       0.88      0.78      0.82        45
     accuracy                            0.85       100
     ```


In [ ]:
def plot_decision_boundary_plotly(model, X, y, title="Decision Boundary"):
    """
    Plot the decision boundary using Plotly. Works for both PyTorch and sklearn models.
    """
    x_min, x_max = X[:, 0].min() - 0.1, X[:, 0].max() + 0.1
    y_min, y_max = X[:, 1].min() - 0.1, X[:, 1].max() + 0.1
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 200),
                         np.linspace(y_min, y_max, 200))

    grid = np.c_[xx.ravel(), yy.ravel()]

    # Prediction logic for PyTorch and scikit-learn models
    if hasattr(model, 'forward'):
        # For PyTorch models: using forward pass
        with torch.no_grad():
            grid_tensor = torch.FloatTensor(grid)
            outputs = model(grid_tensor)
            # Multi-class case
            if outputs.ndim > 1 and outputs.shape[1] > 1:
                Z = np.argmax(outputs.numpy(), axis=1)
            else:
                Z = (outputs.numpy() > 0.5).astype(int).reshape(-1)
    else:
        # For scikit-learn models
        Z = model.predict(grid)

    Z = Z.reshape(xx.shape)

    # Create contour plot with Plotly
    fig = go.Figure()
    fig.add_trace(
        go.Contour(
            x=np.linspace(x_min, x_max, 200),
            y=np.linspace(y_min, y_max, 200),
            z=Z,
            colorscale='Viridis',
            opacity=0.3,
            showscale=False
        )
    )
    # Scatter plot for data points
    fig.add_trace(
        go.Scatter(
            x=X[:, 0],
            y=X[:, 1],
            mode="markers",
            marker=dict(
                color=y,
                colorscale='Viridis',
                line=dict(width=1, color='black')
            )
        )
    )
    fig.update_layout(
        title=title,
        xaxis_title='Feature 1',
        yaxis_title='Feature 2'
    )
    fig.show()

def evaluate_classifier(y_true, y_pred):
    """Print evaluation metrics for a classifier."""
    print("Accuracy:", accuracy_score(y_true, y_pred))
    print("\nConfusion Matrix:")
    print(confusion_matrix(y_true, y_pred))
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred))


---

### **Part 1: Logistic Regression**

In this section, we will cover:

- **Binary Logistic Regression:** using a simple PyTorch model with a sigmoid function.
- **Multiple Logistic Regression:** applying the method on a breast cancer dataset.
- **Multinomial Logistic Regression:** extending logistic regression to handle multi-class cases using the softmax function.


#### **1.1 Binary Logistic Regression with PyTorch**

**Key Concepts:**

- **Sigmoid Function:**  
   $
   \sigma(z) = \frac{1}{1 + e^{-z}}
   $

- **Loss Function:** Binary Cross-Entropy loss is used.

We generate a synthetic dataset (with two features) for binary classification, standardize the features, and define a simple PyTorch model.


In [ ]:
# Generate synthetic data for binary classification
X, y = make_classification(n_samples=200, n_features=2, n_redundant=0,
                           n_informative=2, random_state=42, n_clusters_per_class=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define PyTorch model for logistic regression
class LogisticRegressionModel(nn.Module):
    def __init__(self, input_dim):
        super(LogisticRegressionModel, self).__init__()
        self.linear = nn.Linear(input_dim, 1)

    def forward(self, x):
        return torch.sigmoid(self.linear(x))

# Convert data to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train_scaled)
y_train_tensor = torch.FloatTensor(y_train.reshape(-1, 1))
X_test_tensor = torch.FloatTensor(X_test_scaled)
y_test_tensor = torch.FloatTensor(y_test.reshape(-1, 1))

# Create DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)

# Initialize model, loss, optimizer
input_dim = X_train_scaled.shape[1]
model_binary = LogisticRegressionModel(input_dim)
criterion = nn.BCELoss()
optimizer = optim.SGD(model_binary.parameters(), lr=0.01)

# Training loop
epochs = 1000
for epoch in range(epochs):
    for inputs, labels in train_loader:
        outputs = model_binary(inputs)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluate model on test set
model_binary.eval()
with torch.no_grad():
    y_pred_probs = model_binary(X_test_tensor)
    y_pred = (y_pred_probs > 0.5).float().numpy().flatten()

print("\nBinary Logistic Regression Evaluation:")
evaluate_classifier(y_test, y_pred)

# Plot decision boundary using Plotly
plot_decision_boundary_plotly(model_binary, X_train_scaled, y_train, title="Binary Logistic Regression (Training Data)")
plot_decision_boundary_plotly(model_binary, X_test_scaled, y_test, title="Binary Logistic Regression (Test Data)")


#### **1.2 Multiple Logistic Regression**

Here we use the Breast Cancer dataset (which has multiple features) to demonstrate logistic regression on a real-world, higher-dimensional dataset.


In [ ]:
data = load_breast_cancer()
X_multi = data.data
y_multi = data.target
print(f"Dataset: Breast Cancer Dataset with {X_multi.shape[1]} features")

X_train, X_test, y_train, y_test = train_test_split(X_multi, y_multi, test_size=0.3, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train_scaled)
y_train_tensor = torch.FloatTensor(y_train.reshape(-1, 1))
X_test_tensor = torch.FloatTensor(X_test_scaled)

# Initialize and train model
model_multi = LogisticRegressionModel(X_train_scaled.shape[1])
criterion = nn.BCELoss()
optimizer = optim.SGD(model_multi.parameters(), lr=0.01)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)

epochs = 1000
for epoch in range(epochs):
    for inputs, labels in train_loader:
        outputs = model_multi(inputs)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluate the multiple logistic regression model
model_multi.eval()
with torch.no_grad():
    y_pred_probs = model_multi(X_test_tensor)
    y_pred = (y_pred_probs > 0.5).float().numpy().flatten()

print("\nMultiple Logistic Regression Evaluation:")
evaluate_classifier(y_test, y_pred)


#### **1.3 Multinomial Logistic Regression**

For multiclass classification we define a model using the softmax output. The softmax function is given by:

$
\text{softmax}(z_i) = \frac{e^{z_i}}{\sum_{j} e^{z_j}}
$

We generate a synthetic multi-class dataset and then train a PyTorch model using the cross-entropy loss.


In [ ]:
# Generate synthetic data for multi-class classification
X_multiclass, y_multiclass = make_classification(n_samples=500, n_features=2, n_informative=2,
                                                 n_redundant=0, n_classes=3, n_clusters_per_class=1,
                                                 random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_multiclass, y_multiclass, test_size=0.3, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define a multinomial logistic regression model using softmax
class MultinomialLogisticRegression(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(MultinomialLogisticRegression, self).__init__()
        self.linear = nn.Linear(input_dim, num_classes)

    def forward(self, x):
        return torch.softmax(self.linear(x), dim=1)

# Convert data to tensors
X_train_tensor = torch.FloatTensor(X_train_scaled)
y_train_tensor = torch.LongTensor(y_train)
X_test_tensor = torch.FloatTensor(X_test_scaled)

# Initialize model, loss (cross-entropy) and optimizer
model_multi_class = MultinomialLogisticRegression(X_train_scaled.shape[1], 3)
criterion_multi = nn.CrossEntropyLoss()
optimizer_multi = optim.SGD(model_multi_class.parameters(), lr=0.1)

# Training loop
epochs = 1000
for epoch in range(epochs):
    outputs = model_multi_class(X_train_tensor)
    loss = criterion_multi(outputs, y_train_tensor)

    optimizer_multi.zero_grad()
    loss.backward()
    optimizer_multi.step()

    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluate the multinomial model
model_multi_class.eval()
with torch.no_grad():
    outputs = model_multi_class(X_test_tensor)
    _, y_pred_tensor = torch.max(outputs, 1)
    y_pred = y_pred_tensor.numpy()

print("\nMultinomial Logistic Regression Evaluation:")
evaluate_classifier(y_test, y_pred)

# Plot decision boundary using Plotly
plot_decision_boundary_plotly(model_multi_class, X_test_scaled, y_test, title="Multinomial Logistic Regression")


### **Part 2: LDA and Other Classification Methods**

In this section, we apply classic classification techniques:

- **Linear Discriminant Analysis (LDA)**
- **Quadratic Discriminant Analysis (QDA)**
- **Naive Bayes**

These methods are demonstrated on a synthetic two-feature dataset.


In [ ]:
# Generate and standardize data for LDA/QDA/Naive Bayes
X, y = make_classification(n_samples=200, n_features=2, n_redundant=0,
                           n_informative=2, random_state=42, n_clusters_per_class=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Linear Discriminant Analysis (LDA)
lda = LinearDiscriminantAnalysis()
lda.fit(X_train_scaled, y_train)
y_pred_lda = lda.predict(X_test_scaled)
print("LDA Evaluation:")
evaluate_classifier(y_test, y_pred_lda)
plot_decision_boundary_plotly(lda, X_test_scaled, y_test, title="LDA Decision Boundary")

# Quadratic Discriminant Analysis (QDA)
qda = QuadraticDiscriminantAnalysis()
qda.fit(X_train_scaled, y_train)
y_pred_qda = qda.predict(X_test_scaled)
print("\nQDA Evaluation:")
evaluate_classifier(y_test, y_pred_qda)
plot_decision_boundary_plotly(qda, X_test_scaled, y_test, title="QDA Decision Boundary")

# Naive Bayes
nb = GaussianNB()
nb.fit(X_train_scaled, y_train)
y_pred_nb = nb.predict(X_test_scaled)
print("\nNaive Bayes Evaluation:")
evaluate_classifier(y_test, y_pred_nb)
plot_decision_boundary_plotly(nb, X_test_scaled, y_test, title="Naive Bayes Decision Boundary")


In [ ]:
# Generate and standardize data for LDA/QDA/Naive Bayes
X, y = make_classification(n_samples=200, n_features=2, n_redundant=0,
                           n_informative=2, random_state=42, n_clusters_per_class=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

# Train LDA, QDA, and Naive Bayes models
lda = LinearDiscriminantAnalysis()
lda.fit(X_train_scaled, y_train)

qda = QuadraticDiscriminantAnalysis()
qda.fit(X_train_scaled, y_train)

nb = GaussianNB()
nb.fit(X_train_scaled, y_train)

# Create a common meshgrid over the feature space
x_min, x_max = X_test_scaled[:, 0].min() - 0.1, X_test_scaled[:, 0].max() + 0.1
y_min, y_max = X_test_scaled[:, 1].min() - 0.1, X_test_scaled[:, 1].max() + 0.1
xx, yy = np.meshgrid(np.linspace(x_min, x_max, 200),
                     np.linspace(y_min, y_max, 200))
grid = np.c_[xx.ravel(), yy.ravel()]

# For each classifier, get the probability for class 1 over the grid
# Note: For binary classification, the decision boundary is where P(class=1)=0.5
Z_lda = lda.predict_proba(grid)[:, 1].reshape(xx.shape)
Z_qda = qda.predict_proba(grid)[:, 1].reshape(xx.shape)
Z_nb  = nb.predict_proba(grid)[:, 1].reshape(xx.shape)

# Create combined Plotly figure with contours from each classifier
import plotly.graph_objects as go

fig = go.Figure()

# LDA decision boundary (blue)
fig.add_trace(go.Contour(
    x=np.linspace(x_min, x_max, 200),
    y=np.linspace(y_min, y_max, 200),
    z=Z_lda,
    contours=dict(
        start=0.5,
        end=0.5,
        size=0.01,
        coloring='lines'
    ),
    line=dict(color='blue', width=2),
    # Force the colorscale to be solid blue:
    colorscale=[[0, 'blue'], [1, 'blue']],
    showscale=False,
    name='LDA'
))

# QDA decision boundary (red)
fig.add_trace(go.Contour(
    x=np.linspace(x_min, x_max, 200),
    y=np.linspace(y_min, y_max, 200),
    z=Z_qda,
    contours=dict(
        start=0.5,
        end=0.5,
        size=0.01,
        coloring='lines'
    ),
    line=dict(color='red', width=2),
    colorscale=[[0, 'red'], [1, 'red']],
    showscale=False,
    name='QDA'
))

# Naive Bayes decision boundary (green)
fig.add_trace(go.Contour(
    x=np.linspace(x_min, x_max, 200),
    y=np.linspace(y_min, y_max, 200),
    z=Z_nb,
    contours=dict(
        start=0.5,
        end=0.5,
        size=0.01,
        coloring='lines'
    ),
    line=dict(color='green', width=2),
    colorscale=[[0, 'green'], [1, 'green']],
    showscale=False,
    name='Naive Bayes'
))

# Add scatter plot for the test data
fig.add_trace(go.Scatter(
    x=X_test_scaled[:, 0],
    y=X_test_scaled[:, 1],
    mode='markers',
    marker=dict(
        color=y_test,
        colorscale='Viridis',
        line=dict(width=1, color='black')
    ),
    name='Test Data'
))

fig.update_layout(
    title='Combined Decision Boundaries: LDA (blue), QDA (red), Naive Bayes (green)',
    xaxis_title='Feature 1',
    yaxis_title='Feature 2'
)

fig.show()


### **Part 3: Resampling Methods**

Here we illustrate the following resampling techniques:

- **Leave-One-Out Cross-Validation (LOOCV)**
- **K-Fold Cross-Validation**
- **Bootstrap**

These techniques are useful for assessing model generalization and understanding the $bias-variance \space \space trade-off$.


In [ ]:
# Using the Breast Cancer dataset to demonstrate resampling methods
X, y = load_breast_cancer(return_X_y=True)
print("Dataset shape:", X.shape)

# LOOCV
loocv = LeaveOneOut()
model = LinearDiscriminantAnalysis()
loocv_scores = cross_val_score(model, X, y, cv=loocv, scoring='accuracy')
print(f"\nLOOCV - Mean Accuracy: {loocv_scores.mean():.4f}, Std: {loocv_scores.std():.4f}")

# K-Fold Cross-Validation for different k values
for k in [5, 10]:
    kf = KFold(n_splits=k, shuffle=True, random_state=42)
    cv_scores = cross_val_score(model, X, y, cv=kf, scoring='accuracy')
    print(f"{k}-Fold CV - Mean Accuracy: {cv_scores.mean():.4f}, Std: {cv_scores.std():.4f}")

# Bias-Variance Trade-off demonstration
k_range = [2, 5, 10, 20, len(X)]  # last one is LOOCV
mean_scores = []
std_scores = []
for k in k_range:
    if k == len(X):
        cv = LeaveOneOut()
        label = "LOOCV"
    else:
        cv = KFold(n_splits=k, shuffle=True, random_state=42)
        label = f"{k}-fold"
    cv_scores = cross_val_score(model, X, y, cv=cv, scoring='accuracy')
    mean_scores.append(cv_scores.mean())
    std_scores.append(cv_scores.std())
    print(f"{label} - Mean Accuracy: {cv_scores.mean():.4f}, Std: {cv_scores.std():.4f}")

# Plot bias-variance trade-off with Plotly
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=[str(k) if k != len(X) else "LOOCV" for k in k_range],
        y=mean_scores,
        error_y=dict(type='data', array=std_scores, visible=True),
        mode='lines+markers'
    )
)
fig.update_layout(
    title='Bias-Variance Trade-off in K-Fold Cross-Validation',
    xaxis_title='Number of Folds (k)',
    yaxis_title='Accuracy'
)
fig.show()

# Bootstrap
def bootstrap(X, y, model, n_bootstraps=1000):
    n_samples = len(X)
    scores = []
    for _ in range(n_bootstraps):
        indices = np.random.choice(n_samples, n_samples, replace=True)
        X_boot, y_boot = X[indices], y[indices]
        # Out-of-bag samples
        oob_indices = list(set(range(n_samples)) - set(indices))
        if not oob_indices:
            continue  # skip if no out-of-bag samples (rare)
        X_oob, y_oob = X[oob_indices], y[oob_indices]
        model.fit(X_boot, y_boot)
        scores.append(model.score(X_oob, y_oob))
    return scores

bootstrap_scores = bootstrap(X, y, LinearDiscriminantAnalysis(), n_bootstraps=100)
print(f"\nBootstrap - Mean Accuracy: {np.mean(bootstrap_scores):.4f}, Std: {np.std(bootstrap_scores):.4f}")

# Plot bootstrap distribution using Plotly
fig = px.histogram(bootstrap_scores, nbins=20, title='Bootstrap Distribution of LDA Accuracy')
fig.add_vline(x=np.mean(bootstrap_scores), line_dash="dash", line_color="red",
              annotation_text=f"Mean={np.mean(bootstrap_scores):.4f}")
fig.update_layout(xaxis_title="Accuracy", yaxis_title="Frequency")
fig.show()


## **REFERENCES**

[1] Wolberg, W., Mangasarian, O., Street, N., & Street, W. (1993). Breast Cancer Wisconsin (Diagnostic) [Dataset]. UCI Machine Learning Repository. https://doi.org/10.24432/C5DW2B.

[2] Jurafsky and Martin. (2025). Speech and Language Processing: An Introduction to Natural Language Processing, Computational Linguistics, and Speech Recognition with Language Models, 3rd edition. Ch. 5. Logistic Regression. Online manuscript released January 12, 2025. https://web.stanford.edu/~jurafsky/slp3.